<a href="https://colab.research.google.com/github/Cloud-Course-Group-Phoenix/Project-Pheonix/blob/main/UI/HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports and Package installations

In [1]:
import os, sys

try:
    # Clone the GitHub repository if not already present
    if not os.path.exists("/content/Project-Pheonix"):
        !git clone https://github.com/Cloud-Course-Group-Phoenix/Project-Pheonix.git /content/Project-Pheonix

    # Change directory to project root
    %cd /content/Project-Pheonix

    # Checkout the 'main' branch (or develop if you have one)
    !git fetch origin -q
    !git checkout main -q

    # Add project directory to Python path
    sys.path.append("/content/Project-Pheonix/Logic")

    # Step 5: Install required Python packages (quietly)
    %pip install -q firebase
    %pip install -q -U gradio
    %pip install -q paho-mqtt
    %pip install -q requests beautifulsoup4
    %pip install -q nltk
    %pip install -q plotly
    %pip install -q pandas
    %pip install -q importnb

    print("✅ Setup completed successfully.")
    from importnb import Notebook
    with Notebook():
        import Indexmqtt as indx
        #import SensorDataProcessor as senDatProc
        import CloudDB as dbService
        import Admin as admin
        import SearchService as searchService
        import UserManager as userManager

except Exception as e:
    print("❌ Setup failed:", str(e))

# Clear installation output for cleaner display
from IPython.display import clear_output
clear_output()

In [2]:
# Standard library imports
import gradio as gr
import json
import time
import requests
import re
import sys
import os
from datetime import datetime
import operator

# Third-party imports
from firebase import firebase
import paho.mqtt.client as mqtt
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import plotly.graph_objects as go
import pandas as pd

# Download required NLTK data
nltk.download('stopwords', quiet=True)

True

# To remove later

In [3]:
DBLink = "https://couldproject-a621d-default-rtdb.europe-west1.firebasedatabase.app/"
url = "https://mqtt.org/"


#Admin Panel UI

In [4]:
# Create the admin dashboard UI
def create_admin_dashboard():
    with gr.Blocks(title="Admin Dashboard") as dashboard:
        gr.Markdown("# Admin Dashboard")

        with gr.Tab("Index Management"):
            with gr.Row():
                with gr.Column(scale=2):
                    gr.Markdown("### Top 10 Most Searched Terms")
                    top_terms_output = gr.Dataframe(
                        headers=["Rank", "Term", "Searches"],
                        row_count=10,
                        interactive=False
                    )

                    refresh_top_terms = gr.Button("Refresh Top Terms")

                with gr.Column(scale=3):
                    gr.Markdown("### Index Status")
                    index_status_md = gr.Markdown("")

                    with gr.Row():
                        reindex_button = gr.Button("Re-index Content", variant="primary")
                        refresh_index_status = gr.Button("Refresh Status")

                    index_action_output = gr.Textbox(label="Action Output", lines=2)

        with gr.Tab("Daily tasks"):
            with gr.Row():
                with gr.Column():
                    gr.Markdown("tasks list")
                    tasks_output = gr.Dataframe(headers=["tasks"], row_count="dynamic", interactive=False)

                    refresh_tasks_button = gr.Button("Refresh tasks")

                with gr.Column():
                    new_task_input = gr.Textbox(label="new task")
                    add_task_button = gr.Button("➕ Add task")

                    task_to_remove_dropdown = gr.Dropdown(label="choose taks to remove", choices=[], interactive=True)
                    remove_task_button = gr.Button("🗑️ Delete task")

        # Initialize displayed data on load
        def update_top_terms():
            terms_with_counts = admin.get_top_search_terms()
            data = []
            for i, (term, count) in enumerate(terms_with_counts, 1):
                data.append([i, term, f"{count} Searches"])
            return pd.DataFrame(data, columns=["Rank", "Term", "Searches"])

        def update_index_status():
            status = admin.get_index_status()
            return f"**Word Count:** {status['word_count']}<br>**Page Count:** {status['page_count']}<br>**Last Indexed:** {status['last_indexed']}"


        def update_tasks_list():
            tasks = admin.get_daily_tasks()
            df = pd.DataFrame([[task] for task in tasks], columns=["task"])
            return df, gr.update(choices=tasks)

        def add_task(task):
            admin.add_daily_task(task)
            df, choices = update_tasks_list()
            return df, gr.update(choices=choices), gr.update(value="")

        def remove_task(task):
            admin.remove_daily_task(task)
            return update_tasks_list()

        # Set up event handlers
        refresh_top_terms.click(update_top_terms, outputs=top_terms_output)
        refresh_index_status.click(update_index_status, outputs=index_status_md)
        reindex_button.click(admin.reindex_content, outputs=index_action_output)

        refresh_tasks_button.click(update_tasks_list, outputs=[tasks_output, task_to_remove_dropdown])
        add_task_button.click(add_task, inputs=new_task_input, outputs=[tasks_output, task_to_remove_dropdown,new_task_input])
        remove_task_button.click(remove_task, inputs=task_to_remove_dropdown, outputs=[tasks_output, task_to_remove_dropdown])


        # Initialize the UI
        dashboard.load(update_top_terms, outputs=top_terms_output)
        dashboard.load(update_index_status, outputs=index_status_md)
        dashboard.load(update_tasks_list, outputs=[tasks_output, task_to_remove_dropdown])

        return dashboard

#Search Engine UI

In [5]:
# Create the Gradio interface for the search engine
def create_search_interface():
    with gr.Blocks() as search_interface:
        gr.Markdown("""
        # 🔍 MQTT Documentation Search
        """)

        with gr.Row():
            with gr.Column(scale=6):
                search_input = gr.Textbox(
                    placeholder="Search for MQTT topics, concepts, or features...",
                    label="",
                    show_label=False,
                    lines=1
                )
            with gr.Column(scale=1):
                search_button = gr.Button("🔍 Search", variant="primary", size="lg")

        # Information about search capabilities
        with gr.Accordion("Search Tips", open=False):
            gr.Markdown("""
            This search engine uses word stemming to find related word forms.
            - Searching for "connect" will also find "connecting" and "connection"
            - Try multiple keywords to narrow results (e.g., "mqtt broker")
            - Results are ranked by relevance to your query
            """)

        # Area for search results - using HTML instead of Textbox
        search_results = gr.HTML(
            "<p>🔎 Enter your search terms above</p><p>Search for MQTT related terms like 'broker', 'publish', 'subscribe', etc.</p>"
        )

        # Connect the search button to the search function
        search_button.click(
            fn=searchService.search_word,
            inputs=search_input,
            outputs=search_results
        )

        # Add handling for pressing Enter key in the search box
        search_input.submit(
            fn=searchService.search_word,
            inputs=search_input,
            outputs=search_results
        )

    return search_interface

# Sensor UI

In [6]:
# Get Firebase connection
FBconn = firebase.FirebaseApplication(DBLink, None)

#Fetches data from the database based on the enviorment
def fetch_data(path):
    data = FBconn.get(f'/FakeData/{path}', None) or {}
    if data:
        keys = list(data.keys())
        values = list(data.values())
        readable_times = [datetime.utcfromtimestamp(int(ts)).strftime('%H:%M:%S') for ts in keys]
        dates = [datetime.utcfromtimestamp(int(ts)).strftime('%Y-%m-%d') for ts in keys]
    else:
        keys, values, readable_times, dates = [], [], [], []
    return keys, values, readable_times, dates

data_keys_indoor, data_values_indoor, times_indoor, dates_indoor = fetch_data("indoor")
data_keys_outdoor, data_values_outdoor, times_outdoor, _ = fetch_data("outdoor")


enviorment = ['indoor','outdoor']
sensors = {'indoor':['Distance','Temperature','Humidity','Pressure'] , 'outdoor':['DLIGHT','Temperature','Humidity','Pressure']}
sensor_units_map = {'Temperature': '°C','Humidity': '%','Pressure': 'Pa','Distance': 'mm','DLIGHT': 'lx'}
#create and array for the dates and inital hours

date_hour_set = { datetime.utcfromtimestamp(int(ts)).strftime('%Y-%m-%d,%H') for ts in data_keys_indoor}
dates = sorted(set(date.split(',')[0] for date in date_hour_set))
initial_hours = sorted(set( datetime.utcfromtimestamp(int(ts)).strftime('%H') for ts in data_keys_indoor if datetime.utcfromtimestamp(int(ts)).strftime('%Y-%m-%d') == dates[0]))

#updates Dropbox values
def rs_change(rs):
  return gr.update(choices=sensors[rs], value=sensors[rs][0])

#changes the hours in the dropbox according to the data and the selected date
def date_change(selected_date):
    hours = sorted(set(
        datetime.utcfromtimestamp(int(ts)).strftime('%H')
        for ts in data_keys_indoor
        if datetime.utcfromtimestamp(int(ts)).strftime('%Y-%m-%d') == selected_date
    ))
    return gr.update(choices=hours, value=hours[0] if hours else None)

#gets all the sensor's data in the selected hour and date and returns the average value of the sensor
def get_time_and_date(data_keys, data_values, selected_date, selected_hour, name):
    filtered = [
        (datetime.utcfromtimestamp(int(ts)).strftime('%H:%M:%S'), value.get(name, 0))
        for ts, value in zip(data_keys, data_values)
        if datetime.utcfromtimestamp(int(ts)).strftime('%Y-%m-%d') == selected_date
        and datetime.utcfromtimestamp(int(ts)).strftime('%H') == selected_hour
        and value.get(name, 0) != 0
    ]
    times, values = zip(*filtered) if filtered else ([], [])
    clean_values = [v for v in values if v is not None]
    avg = round(sum(clean_values) / len(clean_values), 2) if clean_values else 0
    return list(times), list(values), avg

# Data visualization
def plot_graph(place, name, date, hour):
    data = {
        'indoor': (data_keys_indoor, data_values_indoor),
        'outdoor': (data_keys_outdoor, data_values_outdoor)
    }.get(place)

    if not all([place, name, date, hour]):
        fig = go.Figure().update_layout(
            title="⚠️ Missing selection: Please choose environment, sensor, date, and hour."
        )
        return fig, "Error: Missing input."

    if not data:
        fig = go.Figure().update_layout(title="No data available")
        return fig, "No average"

    try:
        keys, values = data
        times, values, avg = get_time_and_date(keys, values, date, hour, name)
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=times, y=values, mode='lines', name=name))
        fig.update_layout(
            title=f'Sensor {name} Over Time',
            xaxis_title='Time',
            yaxis_title=f"{name} ({sensor_units_map.get(name, '')})"
        )
        return fig, f"Average: {avg} {sensor_units_map.get(name, '')}"
    except Exception as e:
        fig = go.Figure().update_layout(title=f'Error: {str(e)}')
        return fig, "Error occurred"


####################################

def create_sensor_data_ui():
  with gr.Blocks() as sensor:
      gr.Markdown("## 📊 Sensor Data Visualization")
      with gr.Row():
        with gr.Column(scale=1): pass
        with gr.Column(scale=2):
            rs = gr.Dropdown(choices=enviorment, value='indoor', label="Environment")
            rs_sensors = gr.Dropdown(choices=sensors['indoor'], interactive=True, label="Sensor")
            rs_dates = gr.Dropdown(choices=dates, interactive=True, label='Date')
            rs_hours = gr.Dropdown(choices=initial_hours, interactive=True, label='Hour')
            rs.change(fn=rs_change, inputs=rs, outputs=rs_sensors)
            rs_dates.change(fn=date_change, inputs=rs_dates, outputs=rs_hours)
        with gr.Column(scale=1): pass

      with gr.Row():
        with gr.Column(scale=1): pass
        with gr.Column(scale=2):
            submit_btn = gr.Button("Generate", variant="primary")
        with gr.Column(scale=1): pass

      with gr.Column():
        plot_output = gr.Plot(label="Sensor Data Graph")
        avg_textbox = gr.Textbox(label="Average Value")

      submit_btn.click(
        fn=plot_graph,
        inputs=[rs, rs_sensors, rs_dates, rs_hours],
        outputs=[plot_output, avg_textbox]
      )
  return sensor


#Shop UI

In [13]:
# Function to create the Shop UI
def create_shop_ui(authenticated_user_state):
    # Get the current user's data
    username = authenticated_user_state.value
    user_data = dbService.get_user_by_username(username)
    initial_coins = dbService.get_coins_from_db(username) if user_data else 0

    # Create the Gradio interface
    with gr.Blocks(theme=gr.themes.Citrus()) as shop:
        gr.Markdown("## 🛒 Shop")

        # User greeting and current coin display
        with gr.Row():
            user_greeting = gr.Markdown(f"Welcome **{username}**")
            current_coins = gr.Markdown(f"Coins: **{initial_coins}** 💰")

        # Checkbox group for selecting rewards
        cart = gr.State([])
        items_to_add = gr.CheckboxGroup(
            ["Free Coffee ☕️ :50 coins", "Free Meal 🍔 :100 coins", "Pizza Party 🍕 :200 coins", "Water Park 💧 :300 coins", "Day Off 😄 :400 coins"],
            label="Choose Items to Add"
        )

        with gr.Row():
            add_button = gr.Button("➕ Add Items to Cart", variant="primary", size="lg")
            delete_button = gr.Button("❌ Clear Cart", variant="secondary")
        cart_display = gr.Markdown("🛒 **Cart is empty**")
        cart_size = gr.Number(label="Cart Size", interactive=False)
        checkout_result = gr.Markdown("")


        # Function to update user's coins using userManager
        def update_user_coins(username, new_coins):
            user_data = userManager.get_user_by_username(username)
            if user_data:
                # Update the user in the database
                dbService.insert_coins_to_db(username)
                return True
            return False

        # Returns a string of all items of the cart, or if it's empty returns empty cart
        def format_cart(cart_list):
            if not cart_list:
                return "🛒 **Cart is empty**"
            return "🛒 **Your Cart:**\n" + "\n".join([f"- {item}" for item in cart_list])

        # Adds items to the cart
        def add_items(new_items, previous_cart):
            new_cart = previous_cart + new_items
            return new_cart, format_cart(new_cart), len(new_cart)

        # Initiates checkout, sums the total cost of all items, returns what was bought and subtracts from the user's coins
        def checkout(cart_items):
            if not cart_items:
                return "❌ Your cart is empty!", cart_items, format_cart(cart_items), len(cart_items), f"Coins: **{dbService.get_coins_from_db(username)}** 💰"

            current_coins = dbService.get_coins_from_db(username)
            messages = ["🧾 **Checkout Summary:**"]
            total_cost = 0

            for item in cart_items:
                match item:
                    case "Free Coffee ☕️ :50 coins":
                        messages.append("☕️ Coffee - 50 coins")
                        total_cost += 50
                    case "Free Meal 🍔 :100 coins":
                        messages.append("🍔 Meal - 100 coins")
                        total_cost += 100
                    case "Pizza Party 🍕 :200 coins":
                        messages.append("🍕 Pizza - 200 coins")
                        total_cost += 200
                    case "Water Park 💧 :300 coins":
                        messages.append("💧 Water Park - 300 coins")
                        total_cost += 300
                    case "Day Off 😄 :400 coins":
                        messages.append("😄 Day Off - 400 coins")
                        total_cost += 400
                    case _:
                        messages.append(f"❓ Unknown item: {item}")

            # Checks if the total sum of items in the cart is smaller than the amount of coins the user has
            if total_cost > current_coins:
                return "❌ Not enough coins to complete the purchase!", cart_items, format_cart(cart_items), len(cart_items), f"Coins: **{current_coins}** 💰"

            # Update user's coins
            new_coins = current_coins - total_cost
            if update_user_coins(username, new_coins):
                messages.append(f"\n💰 **Total Cost:** {total_cost} coins")
                messages.append(f"💰 **Remaining Coins:** {new_coins} coins")
                return "\n".join(messages), [], format_cart([]), 0, f"Coins: **{new_coins}** 💰"
            else:
                return "❌ Error updating coins. Please try again.", cart_items, format_cart(cart_items), len(cart_items), f"Coins: **{current_coins}** 💰"

        # Clears the cart
        def delete_cart(cart_items):
            return [], "🛒 **Cart is empty**", 0

        with gr.Row():
            checkout_button = gr.Button("✅ Checkout", variant="secondary")

        # Event handlers
        add_button.click(
            fn=add_items,
            inputs=[items_to_add, cart],
            outputs=[cart, cart_display, cart_size]
        )

        checkout_button.click(
            fn=checkout,
            inputs=[cart],
            outputs=[checkout_result, cart, cart_display, cart_size, current_coins]
        )

        delete_button.click(
            fn=delete_cart,
            inputs=[cart],
            outputs=[cart, cart_display, cart_size]
        )

    return shop

#Unified UI

In [14]:
# Create the main interface with authentication flow
def create_main_interface():
    # Create the interface
    with gr.Blocks(title="Cloud Project - Phoenix", theme=gr.themes.Base()) as main_interface:
        gr.Markdown("# 🐦 Phoenix Team Project")

        # Authentication state variables
        authenticated_user = gr.State(None)
        is_admin = gr.State(False)
        is_logged_in = gr.State(False)

        # Main tabs for unauthenticated users (initially visible)
        with gr.Column(visible=True) as main_tabs:
            with gr.Tabs(selected=0):
                with gr.TabItem("🏠 Home"):
                    gr.Markdown("""
                    # Welcome to Our Phoenix Team Project!

                    ## Available Features:
                    - Search Engine for MQTT
                    - Sensor Data Visualization
                    - Reward Shop
                    - Admin Dashboard (for admin users)

                    Please log in to access all features.
                    """)

                with gr.TabItem("🔍 Search Engine"):
                    create_search_interface()

                with gr.TabItem("🔐 Login"):
                    gr.Markdown("## 🔐 Login")
                    with gr.Row():
                        login_username = gr.Textbox(label="Username")
                        login_password = gr.Textbox(label="Password", type="password")
                    login_button = gr.Button("Login", variant="primary")
                    login_output = gr.Textbox(label="Login Status", lines=1)

        # Admin tabs (initially hidden)
        with gr.Column(visible=False) as admin_tabs:
            with gr.Tabs(selected=0):
                with gr.TabItem("🏠 Home"):
                    gr.Markdown("""
                    # Welcome Admin! 🔥

                    You have successfully logged in as an **Administrator**.

                    ## Admin Features Available:
                    - 👨‍💼 **Admin Dashboard** - Manage search terms, index status, and daily tasks
                    - 📝 **User Registration** - Create new user accounts and admin accounts
                    - 🔍 **Search Engine** - Full access to MQTT documentation search
                    - 📊 **Sensor Data** - View and analyze sensor data visualizations
                    - 🛒 **Shop** - Access the reward shop system


                    Select any tab above to get started!
                    """)

                with gr.TabItem("👨‍💼 Admin Dashboard"):
                    create_admin_dashboard()

                with gr.TabItem("🔍 Search Engine"):
                    create_search_interface()

                with gr.TabItem("📊 Sensor Data"):
                    create_sensor_data_ui()

                with gr.TabItem("🛒 Shop"):
                    create_shop_ui(authenticated_user)

                with gr.TabItem("📝 Register New Users"):
                    gr.Markdown("## 📝 Register New User (Admin Only)")
                    with gr.Row():
                        reg_username = gr.Textbox(label="Username")
                        reg_password = gr.Textbox(label="Password", type="password")
                        reg_confirm_password = gr.Textbox(label="Confirm Password", type="password")
                    reg_is_admin = gr.Checkbox(label="Register as Admin")
                    register_button = gr.Button("Register User", variant="primary")
                    register_output = gr.Textbox(label="Registration Status", lines=1)

        # Regular user tabs (initially hidden)
        with gr.Column(visible=False) as user_tabs:
            with gr.Tabs(selected=0):
                with gr.TabItem("🏠 Home"):
                    gr.Markdown("""
                    # Welcome User! 🎉

                    You have successfully logged in to the Phoenix Team Project.

                    ## Available Features:
                    - 🔍 **Search Engine** - Search through MQTT documentation with advanced word stemming
                    - 📊 **Sensor Data** - Visualize indoor and outdoor sensor data with interactive charts
                    - 🛒 **Shop** - Use your coins to purchase rewards and benefits

                    ## How to Get Started:
                    1. **Search Engine**: Try searching for terms like "broker", "publish", or "subscribe"
                    2. **Sensor Data**: Select environment, sensor type, date, and hour to view data
                    3. **Shop**: Browse available rewards and add items to your cart

                    ## Tips:
                    - The search engine uses word stemming (searching "connect" finds "connection", "connecting")
                    - Sensor data is updated in real-time from our IoT devices
                    - You start with 2500 coins in the shop system

                    Select any tab above to explore the features!
                    """)

                with gr.TabItem("🔍 Search Engine"):
                    create_search_interface()

                with gr.TabItem("📊 Sensor Data"):
                    create_sensor_data_ui()

                with gr.TabItem("🛒 Shop"):
                    create_shop_ui(authenticated_user)

        # Logout section (initially hidden)
        with gr.Row(visible=False) as logout_section:
            logout_button = gr.Button("🔓 Logout", variant="secondary", size="lg")

        # Event handler functions
        def handle_login(username, password):
            message, user, admin_status = userManager.login(username, password)
            if user is not None:
                if admin_status:
                    return (
                        message, user, admin_status, True,
                        gr.update(visible=False), gr.update(visible=True),
                        gr.update(visible=False), gr.update(visible=True)
                    )
                else:
                    return (
                        message, user, admin_status, True,
                        gr.update(visible=False), gr.update(visible=False),
                        gr.update(visible=True), gr.update(visible=True)
                    )
            else:
                return (
                    message, None, False, False,
                    gr.update(visible=True), gr.update(visible=False),
                    gr.update(visible=False), gr.update(visible=False)
                )

        def handle_register(username, password, confirm, is_admin_val):
            message, user, admin_status = userManager.register_account(username, password, confirm, is_admin_val)
            return message

        def handle_logout():
            return (
                None, False, False,
                gr.update(visible=True), gr.update(visible=False),
                gr.update(visible=False), gr.update(visible=False),
                gr.update(value=""), gr.update(value=""), gr.update(value="")
            )

        # Event handlers
        login_button.click(
            fn=handle_login,
            inputs=[login_username, login_password],
            outputs=[login_output, authenticated_user, is_admin, is_logged_in,
                    main_tabs, admin_tabs, user_tabs, logout_section]
        )

        register_button.click(
            fn=handle_register,
            inputs=[reg_username, reg_password, reg_confirm_password, reg_is_admin],
            outputs=[register_output]
        )

        logout_button.click(
            fn=handle_logout,
            outputs=[authenticated_user, is_admin, is_logged_in,
                    main_tabs, admin_tabs, user_tabs, logout_section,
                    login_username, login_password, login_output]
        )

    return main_interface

# Create the main interface
main_interface = create_main_interface()
main_interface.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://4a732c3d5c0d83b916.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7862 <> https://4a732c3d5c0d83b916.gradio.live
